In [18]:
import os
import pickle
import json
import numpy as np
from keras.layers import Input,Dense,Conv2D,Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.models import Model
from sklearn.metrics import f1_score
import pickle

In [3]:
with open('inception_results.pckl','rb') as inception_res:
    inception_val,true_labels,inception_test_val = pickle.load(inception_res)
with open('nasnet_results.pckl','rb') as nasnet_res:
    nasnet_val,true_labels,nasnet_test_val = pickle.load(nasnet_res)
with open('resnet_results.pckl','rb') as resnet_res:
    resnet_val,true_labels,resnet_test_val = pickle.load(resnet_res)
with open('vgg_results.pckl','rb') as vgg_res:
    vgg_val,true_labels,vgg_test_val = pickle.load(vgg_res)

In [8]:
number_of_models = 4
number_of_classes = 229

In [30]:
input_data = np.dstack((inception_val,nasnet_val,resnet_val,vgg_val))

In [42]:
input_dt = np.expand_dims(input_data,axis=3)

In [44]:
inp = Input(shape=(number_of_classes,number_of_models,1)) 

In [45]:
middle = Conv2D(16, (1, 2))(inp)
middle = Conv2D(32, (1, 3))(middle)
middle = Flatten()(middle)
middle = Dense(1024)(middle)
final = Dense(number_of_classes, activation='sigmoid')(middle)
model = Model(inputs= inp,outputs=final)

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 229, 4, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 229, 3, 16)        48        
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 229, 1, 32)        1568      
_________________________________________________________________
flatten_7 (Flatten)          (None, 7328)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              7504896   
_________________________________________________________________
dense_13 (Dense)             (None, 229)               234725    
Total params: 7,741,237
Trainable params: 7,741,237
Non-trainable params: 0
_________________________________________________________________


In [94]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
for train,test in kf.split(input_dt):
    model.compile(optimizer='adam',loss='binary_crossentropy')
    model.fit(input_dt[train],np.array(true_labels)[train],epochs=5)
    predicted = model.predict(input_dt[test])
    predicted[predicted>=0.3] = 1
    predicted[predicted<0.3] = 0
    print(f1_score(np.array(np.array(true_labels)[test]),predicted,average='micro'))

Epoch 1/5
7917/7917 [==============================] - 3s 410us/step - loss: 0.0630
Epoch 2/5
7917/7917 [==============================] - 3s 368us/step - loss: 0.0616
Epoch 3/5
7917/7917 [==============================] - 3s 369us/step - loss: 0.0606
Epoch 4/5
7917/7917 [==============================] - 3s 368us/step - loss: 0.0598
Epoch 5/5
7917/7917 [==============================] - 3s 369us/step - loss: 0.0593
0.6089010442565888
Epoch 1/5
7917/7917 [==============================] - 3s 405us/step - loss: 0.0608
Epoch 2/5
7917/7917 [==============================] - 3s 370us/step - loss: 0.0594
Epoch 3/5
7917/7917 [==============================] - 3s 378us/step - loss: 0.0586
Epoch 4/5
7917/7917 [==============================] - 3s 373us/step - loss: 0.0579
Epoch 5/5
7917/7917 [==============================] - 3s 406us/step - loss: 0.0573
0.6131916547881301
Epoch 1/5
7918/7918 [==============================] - 4s 450us/step - loss: 0.0594
Epoch 2/5
7918/7918 [=================

In [95]:
predicted = model.predict(input_dt)
predicted[predicted>=0.3] = 1
predicted[predicted<0.3] = 0
print(f1_score(np.array(true_labels),predicted,average='micro'))

0.6848945607056917


In [96]:
predicted = model.predict(np.expand_dims(np.dstack((inception_test_val,nasnet_test_val,resnet_test_val,vgg_test_val)),axis=3))
predicted[predicted>=0.3] = 1
predicted[predicted<0.3] = 0

In [97]:
outputs = []
for i in range(predicted.shape[0]):
        arr = np.where(predicted[i]==1)[0]
        outputs.append(" ".join(map(str, arr.tolist())))

In [98]:
import pandas as pd
df = pd.DataFrame()

In [99]:
with open("test_data_vgg19.pckl",'rb') as write_file:
    image_features,img_ids = pickle.load(write_file)

In [100]:
df['image_id'] = img_ids
df['label_id'] = outputs

In [ ]:
df.to_csv("predictions.csv",index=False)